<a href="https://colab.research.google.com/github/Johno4308/ML_googleColab/blob/main/Free_Code_Camp_ML_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

FREE CODE CAMP:
WHERE THE DATASET WAS FOUND
The data are MC generated (see below) to simulate registration of high energy gamma particles in a ground-based atmospheric Cherenkov gamma telescope using the imaging technique. Cherenkov gamma telescope observes high energy gamma rays, taking advantage of the radiation emitted by charged particles produced inside the electromagnetic showers initiated by the gammas, and developing in the atmosphere.
https://archive.ics.uci.edu/dataset/159/magic+gamma+telescope

In [ ]:
cols = ["fLength", "fWidth", "fSize", "fConc", "fAsym", "fM3Loong", "fM3Trans", "fAplha", "fDist", "class"]
df = pd.read_csv('magic04.data', names=cols)
df.head() #head is first 5 things

,fLength,fWidth,fSize,fConc,fAsym,fM3Loong,fM3Trans,fAplha,fDist,class
28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


In [ ]:
df["class"] = (df["class"] == "g").astype(int)

In [ ]:
df.head()

,fLength,fWidth,fSize,fConc,fAsym,fM3Loong,fM3Trans,fAplha,fDist,class
28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,1
31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,1
162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,1
23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,1
75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,1


In [ ]:
# checking for comminality in gamma fLength column
for label in cols[:-1]:
  plt.hist(df[df["class"]==1][label], color='blue', label='gamma', alpha=0.7, density=True)
  plt.hist(df[df["class"]==0][label], color='red', label='hadron', alpha=0.7, density=True)
  plt.title(label)
  plt.ylabel("Probability")
  plt.xlabel(label)
  plt.legend()
  plt.show()

#Train, validation, test datasets

In [ ]:
# section out the data into the three categories
# 60% for test, 20% for valid (60 to 80), and 20% for test (the rest)
# print(df.shape)
df = df.sample(frac=1).reset_index(drop=True)
train, valid, test = np.split(df, [int(0.6*len(df)), int(0.8*len(df))])

In [ ]:
# scaling the columns, making the function
def scale_dataset(dataframe, oversample=False):
  X = dataframe[dataframe.columns[:-1]].values
  y = dataframe[dataframe.columns[-1]].values

  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  # to resample as there are more gamma then hadron
  # basically keep sampling for the less class in increase the size of our data set
  if oversample:
    ros = RandomOverSampler()
    X, y = ros.fit_resample(X, y)

  data = np.hstack((X, np.reshape(y, (-1, 1))))


  return data, X, y

In [ ]:
# seeing how many of each
print(len(train[train["class"]==1])) #gamma
print(len(train[train["class"]==0])) #hadron

7416
3996


In [ ]:
train, X_train, y_train = scale_dataset(train, oversample=True)
valid, X_valid, y_valid = scale_dataset(valid, oversample=False)
test, X_test, y_test = scale_dataset(test, oversample=False)

In [ ]:
# the effects for RandomOverSampler
len(y_train)

14690

In [ ]:
sum(y_train == 1)

7345

In [ ]:
sum(y_train == 0)

7345

# K NEAREST NEIGHBORS

In [ ]:
# define a distance function - euclidean distance
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [ ]:
# the argument is how many neighbors we want to use
knn_model = KNeighborsClassifier(n_neighbors=1)
knn_model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=1)

In [ ]:
y_pred = knn_model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.64      0.70      1344
           1       0.82      0.89      0.86      2460

    accuracy                           0.80      3804
   macro avg       0.79      0.77      0.78      3804
weighted avg       0.80      0.80      0.80      3804



# MODEL - NAIVE BAYES RULE
# Formula: P(A|B) = (P(B|A) * P(A)) / P(B)
Prior Probability (P(A)): It represents the probability of an event A occurring without considering any additional evidence.

Likelihood (P(B|A)): It denotes the probability of observing evidence B given that event A has occurred. In other words, it quantifies how likely the evidence is if the event is true.

Posterior Probability (P(A|B)): It is the probability of the event A happening given the evidence B. This is what we want to calculate using Naive Bayes.

Marginal Likelihood (P(B)): It represents the probability of observing the evidence B, irrespective of any specific event.

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
nb_model = GaussianNB()
nb_model = nb_model.fit(X_train, y_train)

In [ ]:
y_pred = nb_model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.43      0.54      1339
           1       0.75      0.90      0.82      2465

    accuracy                           0.74      3804
   macro avg       0.73      0.67      0.68      3804
weighted avg       0.73      0.74      0.72      3804

